In [4]:
""" 
Prepare MIMIC dataset to run deep patient on the dataset
Desired columns: Patient ID, gender, list of ICD codes, mortality
TODO: add more response columns: e.g. number of days in ICU
"""
import getpass
user_id = getpass.getuser()

import sys
sys.path.append(f"/home/{user_id}/OTTEHR/")

import collections
import pandas as pd
import sys
from datetime import datetime
import matplotlib.pyplot as plt
from mimic_common import *
import json

In [8]:
""" 
Global parameters
"""
target_diagnosis = "LUNG CA"

In [ ]:
"""
Read in ADMISSIONS.csv and construct (1) patient ID to admission ID map, and \
    (2) admission ID to duration in hospital map (in seconds)
"""

pid_admids = {}
admid_duration = {}
admid_diagnosis = {}
admid_type = {}

admission_file = open("../mimic_iii/ADMISSIONS.csv", 'r')
admission_file.readline()
for line in admission_file:
    tokens = line.strip().split(',')
    pid = int(tokens[1])
    admid = int(tokens[2])
    diagnosis = tokens[-3]
    adm_type = tokens[6][1:-1]
    if pid in pid_admids: pid_admids[pid].append(admid)
    else: pid_admids[pid] = [admid]

    admit_time = datetime.strptime(tokens[3], '%Y-%m-%d %H:%M:%S')
    discharge_time = datetime.strptime(tokens[4], '%Y-%m-%d %H:%M:%S')
    admid_duration[admid] = (discharge_time-admit_time).total_seconds()
    admid_diagnosis[admid] = diagnosis[1:-1]
    admid_type[admid] = adm_type

admission_file.close()


In [ ]:
""" 
Read in PATIENTS.csv and construct admission ID to gender map
"""
admid_gender = {}
patient_file = open("../mimic_iii/PATIENTS.csv", 'r')
patient_file.readline()
for line in patient_file:
    tokens = line.strip().split(',')
    pid = int(tokens[1])
    gender = str(tokens[2])
    admids = pid_admids[pid]
    for admid in admids:
        admid_gender[admid] = gender[1]

patient_file.close()

In [ ]:
""" 
Consruct admission ID to ICD codes mapping
"""

def convert_to_icd9(dxStr):
    """ 
    Source: https://github.com/mp2893/med2vec/blob/master/process_mimic.py
    I am not sure why ICD codes are converted in this way
    """
    if dxStr.startswith('E'):
        if len(dxStr) > 4: 
            return dxStr[:4] + '.' + dxStr[4:]
        else: 
            return dxStr
    else:
        if len(dxStr) > 3: return dxStr[:3] + '.' + dxStr[3:]
        else: return dxStr

admid_codes = {}
diagnosis_file = open("../mimic_iii/DIAGNOSES_ICD.csv", 'r')
diagnosis_file.readline()
for line in diagnosis_file: # read ADMISSIONS.CSV in order
    tokens = line.strip().split(',')
    admid = int(tokens[2])
    code = tokens[4][1:-1]
    # dxStr = 'D_' + convert_to_icd9(tokens[4][1:-1]) # 1:-1 to remove quotes

    if admid in admid_codes: 
        admid_codes[admid].append(code)
    else: 
        admid_codes[admid] = [code]

diagnosis_file.close()


In [ ]:
""" 
Construct a dataframe to store all information including
- patient ID (index key)
- gender
- expire (mortality)
- list of ICD codes
"""

admid_diagnosis_df = pd.DataFrame(columns=['admid', 'adm_type', 'gender','ICD codes','duration', 'diagnosis', 'label'])

for admid, codes in admid_codes.items():
    new_row = {"admid": admid, "adm_type": admid_type[admid], "gender": admid_gender[admid], \
               "duration": admid_duration[admid], "ICD codes": codes, "diagnosis": admid_diagnosis[admid]}
    admid_diagnosis_df = pd.concat([admid_diagnosis_df, pd.DataFrame([new_row])], ignore_index=True)


admid_diagnosis_df = admid_diagnosis_df.set_index('admid')

In [ ]:
# Transform ICD code to standard ICD-9 code

""" 
Consruct admission ID to ICD codes mapping
"""

def convert_to_icd9(dxStr):
    """ 
    Adapted from https://github.com/mp2893/med2vec/blob/master/process_mimic.py
    """
    if dxStr.startswith('E'):
        if len(dxStr) > 4: return dxStr[:4] + '.' + dxStr[4:]
        else: return dxStr
    else:
        if len(dxStr) > 3: return dxStr[:3] + '.' + dxStr[3:]
        else: return dxStr



In [14]:
# convert ICD to code to ICD-9
diagnose_path = f"/home/{user_id}/OTTEHR/mimic_exp/mimic_iii/D_ICD_DIAGNOSES.csv"
diagnose_df = pd.read_csv(diagnose_path, header=0, index_col=None)
diagnose_df

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...,...,...
14562,14432,V7399,Scrn unspcf viral dis,Special screening examination for unspecified ...
14563,14433,V740,Screening for cholera,Screening examination for cholera
14564,14434,V741,Screening-pulmonary TB,Screening examination for pulmonary tuberculosis
14565,14435,V742,Screening for leprosy,Screening examination for leprosy (Hansen's di...


In [20]:
# only run once
new_codes = []
for _, row in diagnose_df.iterrows():
    new_codes.append(convert_to_icd9(row['ICD9_CODE']))
diagnose_df['ICD code'] = new_codes
diagnose_df = diagnose_df.set_index('ICD code')
print(diagnose_df)
diagnose_df.to_csv(diagnose_path, index=True, header=True)

          ROW_ID ICD9_CODE               SHORT_TITLE  \
ICD code                                               
011.66       174     01166     TB pneumonia-oth test   
011.70       175     01170    TB pneumothorax-unspec   
011.71       176     01171   TB pneumothorax-no exam   
011.72       177     01172  TB pneumothorx-exam unkn   
011.73       178     01173  TB pneumothorax-micro dx   
...          ...       ...                       ...   
V73.99     14432     V7399     Scrn unspcf viral dis   
V74.0      14433      V740     Screening for cholera   
V74.1      14434      V741    Screening-pulmonary TB   
V74.2      14435      V742     Screening for leprosy   
V74.3      14436      V743  Screening for diphtheria   

                                                 LONG_TITLE  
ICD code                                                     
011.66    Tuberculous pneumonia [any form], tubercle bac...  
011.70                Tuberculous pneumothorax, unspecified  
011.71    Tuberculous p

In [ ]:
from ast import literal_eval

admid_diagnosis_path = "../../outputs/mimic/admission_patient_diagnosis_ICD.csv"
admid_diagnosis_df = pd.read_csv(admid_diagnosis_path, index_col=0, header=0)
admid_diagnosis_df['ICD codes'] = admid_diagnosis_df['ICD codes'].apply(literal_eval)
admid_diagnosis_df

In [ ]:
""" 
Add target diagnosis label to admid_diagnosis_df
"""
for index, row in admid_diagnosis_df.iterrows():
    admid_diagnosis_df.at[index, 'ICD codes'] = [convert_to_icd9(code) for code in row['ICD codes']]
    if target_diagnosis in row['diagnosis']:
        admid_diagnosis_df.at[index, 'label'] = 1
    else:
        admid_diagnosis_df.at[index, 'label'] = 0
        
admid_diagnosis_df.to_csv("../../outputs/mimic/admission_patient_diagnosis_ICD.csv", header=True, index=True)
admid_diagnosis_df

In [ ]:
""" 
Choose lung cancer to be the response
Filter out rows which does not contain target diagnosis in diagnosis
"""
admid_diagnosis_target_df = admid_diagnosis_df.loc[admid_diagnosis_df['diagnosis'].str.contains(target_diagnosis)]
admid_diagnosis_target_df 

In [ ]:
""" 
Add target diagnosis label to admid_diagnosis_df
"""
for index, row in admid_diagnosis_df.iterrows():
    if target_diagnosis in row['diagnosis']:
        admid_diagnosis_df.at[index, 'label'] = 1
    else:
        admid_diagnosis_df.at[index, 'label'] = 0

In [ ]:
admid_diagnosis_df.to_csv("../../outputs/mimic/admission_patient_diagnosis_ICD.csv", header=True, index=True)
admid_diagnosis_df

In [5]:
# Run this to add more information to ADMID_DIADNOSIS.csv
admid_diagnosis_path = os.path.join(mimic_output_dir, "admission_patient_diagnosis_ICD.csv")
admid_diagnosis_df = pd.read_csv(admid_diagnosis_path, header=0, index_col=0)
admid_diagnosis_df

,adm_type,gender,ICD codes,duration,diagnosis,label
admid,,,,,,
172335,EMERGENCY,F,"['403.01', '486', '582.81', '585.5', '425.4', ...",530460.0,LEG PAIN,0
173633,EMERGENCY,F,"['403.01', '585.6', '583.81', '710.0', '558.9'...",574560.0,ABDOMINAL PAIN,0
174105,EMERGENCY,M,"['531.00', '410.71', '285.9', '414.01', '725']",425460.0,GASTROINTESTINAL BLEED,0
109976,EMERGENCY,M,"['191.5', '331.4', '530.81']",1284240.0,HYDROCEPHALUS,0
178393,ELECTIVE,M,"['414.01', '411.1', '482.83', '285.9', '272.0'...",485280.0,USA/SDA,0
...,...,...,...,...,...,...
172304,ELECTIVE,F,"['202.80', '348.5', '784.3', '401.9', '272.0',...",449700.0,LEFT BRAIN TUMOR/SDA,0
152542,EMERGENCY,M,"['566', '250.62', '357.2', 'V58.67', '427.31',...",415740.0,PERIRECTAL ABSCESS,0
161999,EMERGENCY,M,"['434.11', '348.5', '348.4', '430', '348.30', ...",692940.0,STROKE;TELEMETRY,0


In [8]:
admission_path = os.path.join(mimic_data_dir, "ADMISSIONS.csv")
admission_df = pd.read_csv(admission_path, header=0, index_col='HADM_ID')
admission_df

,ROW_ID,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
HADM_ID,,,,,,,,,,,,,,,,,,
165315,21,22,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
152223,22,23,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
124321,23,23,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
161859,24,24,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
129635,25,25,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191113,58594,98800,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1
101071,58595,98802,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1
122631,58596,98805,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,RENAL CANCER/SDA,0,1


In [12]:
admission_locations = []
insurances = []
languages = []
religions = []
marital_statuses = []
ethnicities = []
for index, _ in admid_diagnosis_df.iterrows():
    admission_locations.append(admission_df.at[index, 'ADMISSION_LOCATION'])
    insurances.append(admission_df.at[index, 'INSURANCE'])
    languages.append(admission_df.at[index, 'LANGUAGE'])
    religions.append(admission_df.at[index, 'RELIGION'])
    marital_statuses.append(admission_df.at[index, 'MARITAL_STATUS'])
    ethnicities.append(admission_df.at[index, 'ETHNICITY'])

admid_diagnosis_df['admission location'] = admission_locations
admid_diagnosis_df['insurance'] = insurances
admid_diagnosis_df['language'] = languages
admid_diagnosis_df['religion'] = religions
admid_diagnosis_df['marital status'] = marital_statuses
admid_diagnosis_df['ethnicity'] = ethnicities
admid_diagnosis_df.to_csv(admid_diagnosis_path, header=True, index=True)
